# Internet checksum

Este notebook explica cómo se calcula el checksum que utilizan las cabeceras de IPv4, ICMPv4, IGMP, UDP o TCP. Se le conoce habitualmente como «checksum de Internet» o *Internet checksum*.

El checksum permite detectar cambios (incluso de un solo bit) en la transmisión del mensaje. Consiste en una suma de 16 en complemento a uno.

## ¿Cómo funciona?

Partimos de una cabecera IP que encapsula un mensaje ICMP. Inicialmente el campo checksum debe contener un 0.

In [ ]:
ip_header = [
    0x45, 0x00,             # Version, IHL, Type of Service
    0x00, 0x54,             # Total Length (84 bytes)
    0x00, 0x00,             # Identification
    0x40, 0x00,             # Flags, Fragment Offset
    0x40, 0x01,             # TTL, Protocol (ICMP)
    0x00, 0x00,             # Header checksum
    0xC0, 0xA8, 0x00, 0x01, # Source IP (192.168.0.1)
    0xC0, 0xA8, 0x00, 0xC7  # Destination IP (192.168.0.199)
]

Se divide el mensaje (la cabecera IP en este caso) en palabras de 16 bits y se suman tratando el  desbordamiento con la técnica de complemento. Es decir, si la suma supera 0xFFFF el desbordamiento se suma al resultado. Por ejemplo, si se suma 0xFFFE + 4 = 0x10002, se convierte en 0x0002 + 1 = 0x0003.

Aplicado a la cabecera anterior:

In [ ]:
words = [
    0x4500,
    0x0054,
    0x0000,
    0x4000,
    0x4001,
    0x0000,
    0xC0A8,
    0x0001,
    0xC0A8,
    0x00C7
]

In [ ]:
0x4500 + 0x0054 = 0x4554
0x4554 + 0x0000 = 0x4554
0x4554 + 0x4000 = 0x8554
0x8554 + 0x4001 = 0xC555
0xC555 + 0x0000 = 0xC555
0xC555 + 0xC0A8 = 0x185FD
0x85FD + 0x1 = 0x85FE
0x85FE + 0x0001 = 0x85FF
0x85FF + 0xC0A8 = 0x146A7
0x46A7 + 0x1 = 0x46A8
0x46A8 + 0x00C7 = 0x4770

Se calcula el complemento a 1:

In [3]:
hex(~0x4770 & 0xFFFF)

'0xb88f'

Este es el valor que se debe colocar en el campo checksum del mensaje.

Al recibir el mensaje para verificar el checksum, se repite el calculo (incluyendo el checksum) y el resultado debe ser `0xFFFF`.